# Working on...

In [1]:
import copy
import commons
from einops import rearrange, einsum
import math
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
# The following codes is SynthesizerTrn in VITS, let analyze it.
"""
class SynthesizerTrn(nn.Module):

  def __init__(self, 
    n_vocab,
    spec_channels,
    segment_size,
    inter_channels,
    hidden_channels,
    filter_channels,
    n_heads,
    n_layers,
    kernel_size,
    p_dropout,
    resblock, 
    resblock_kernel_sizes, 
    resblock_dilation_sizes, 
    upsample_rates, 
    upsample_initial_channel, 
    upsample_kernel_sizes,
    n_speakers=0,
    gin_channels=0,
    use_sdp=True,
    **kwargs):

    super().__init__()
    self.n_vocab = n_vocab
    self.spec_channels = spec_channels
    self.inter_channels = inter_channels
    self.hidden_channels = hidden_channels
    self.filter_channels = filter_channels
    self.n_heads = n_heads
    self.n_layers = n_layers
    self.kernel_size = kernel_size
    self.p_dropout = p_dropout
    self.resblock = resblock
    self.resblock_kernel_sizes = resblock_kernel_sizes
    self.resblock_dilation_sizes = resblock_dilation_sizes
    self.upsample_rates = upsample_rates
    self.upsample_initial_channel = upsample_initial_channel
    self.upsample_kernel_sizes = upsample_kernel_sizes
    self.segment_size = segment_size
    self.n_speakers = n_speakers
    self.gin_channels = gin_channels
    
    self.use_sdp = use_sdp

    self.enc_p = TextEncoder(n_vocab,
        inter_channels,
        hidden_channels,
        filter_channels,
        n_heads,
        n_layers,
        kernel_size,
        p_dropout)
    self.dec = Generator(inter_channels, resblock, resblock_kernel_sizes, resblock_dilation_sizes, upsample_rates, upsample_initial_channel, upsample_kernel_sizes, gin_channels=gin_channels)
    self.enc_q = PosteriorEncoder(spec_channels, inter_channels, hidden_channels, 5, 1, 16, gin_channels=gin_channels)
    self.flow = ResidualCouplingBlock(inter_channels, hidden_channels, 5, 1, 4, gin_channels=gin_channels)

    if use_sdp:
      self.dp = StochasticDurationPredictor(hidden_channels, 192, 3, 0.5, 4, gin_channels=gin_channels)
    else:
      self.dp = DurationPredictor(hidden_channels, 256, 3, 0.5, gin_channels=gin_channels)

    if n_speakers > 1:
      self.emb_g = nn.Embedding(n_speakers, gin_channels)

  def forward(self, x, x_lengths, y, y_lengths, sid=None):

    x, m_p, logs_p, x_mask = self.enc_p(x, x_lengths)
    if self.n_speakers > 0:
      g = self.emb_g(sid).unsqueeze(-1) # [b, h, 1]
    else:
      g = None

    z, m_q, logs_q, y_mask = self.enc_q(y, y_lengths, g=g)
    z_p = self.flow(z, y_mask, g=g)

    with torch.no_grad():
      # negative cross-entropy 
      s_p_sq_r = torch.exp(-2 * logs_p) # [b, d, t]
      neg_cent1 = torch.sum(-0.5 * math.log(2 * math.pi) - logs_p, [1], keepdim=True) # [b, 1, t_s]
      neg_cent2 = torch.matmul(-0.5 * (z_p ** 2).transpose(1, 2), s_p_sq_r) # [b, t_t, d] x [b, d, t_s] = [b, t_t, t_s]
      neg_cent3 = torch.matmul(z_p.transpose(1, 2), (m_p * s_p_sq_r)) # [b, t_t, d] x [b, d, t_s] = [b, t_t, t_s]
      neg_cent4 = torch.sum(-0.5 * (m_p ** 2) * s_p_sq_r, [1], keepdim=True) # [b, 1, t_s]
      neg_cent = neg_cent1 + neg_cent2 + neg_cent3 + neg_cent4

      attn_mask = torch.unsqueeze(x_mask, 2) * torch.unsqueeze(y_mask, -1)
      attn = monotonic_align.maximum_path(neg_cent, attn_mask.squeeze(1)).unsqueeze(1).detach()

    w = attn.sum(2)
    if self.use_sdp:
      l_length = self.dp(x, x_mask, w, g=g)
      l_length = l_length / torch.sum(x_mask)
    else:
      logw_ = torch.log(w + 1e-6) * x_mask
      logw = self.dp(x, x_mask, g=g)
      l_length = torch.sum((logw - logw_)**2, [1,2]) / torch.sum(x_mask) # for averaging 

    # expand prior
    m_p = torch.matmul(attn.squeeze(1), m_p.transpose(1, 2)).transpose(1, 2)
    logs_p = torch.matmul(attn.squeeze(1), logs_p.transpose(1, 2)).transpose(1, 2)

    z_slice, ids_slice = commons.rand_slice_segments(z, y_lengths, self.segment_size)
    o = self.dec(z_slice, g=g)
    return o, l_length, attn, ids_slice, x_mask, y_mask, (z, z_p, m_p, logs_p, m_q, logs_q)

  def infer(self, x, x_lengths, sid=None, noise_scale=1, length_scale=1, noise_scale_w=1., max_len=None):
    x, m_p, logs_p, x_mask = self.enc_p(x, x_lengths)
    if self.n_speakers > 0:
      g = self.emb_g(sid).unsqueeze(-1) # [b, h, 1]
    else:
      g = None

    if self.use_sdp:
      logw = self.dp(x, x_mask, g=g, reverse=True, noise_scale=noise_scale_w)
    else:
      logw = self.dp(x, x_mask, g=g)
    w = torch.exp(logw) * x_mask * length_scale
    w_ceil = torch.ceil(w)
    y_lengths = torch.clamp_min(torch.sum(w_ceil, [1, 2]), 1).long()
    y_mask = torch.unsqueeze(commons.sequence_mask(y_lengths, None), 1).to(x_mask.dtype)
    attn_mask = torch.unsqueeze(x_mask, 2) * torch.unsqueeze(y_mask, -1)
    attn = commons.generate_path(w_ceil, attn_mask)

    m_p = torch.matmul(attn.squeeze(1), m_p.transpose(1, 2)).transpose(1, 2) # [b, t', t], [b, t, d] -> [b, d, t']
    logs_p = torch.matmul(attn.squeeze(1), logs_p.transpose(1, 2)).transpose(1, 2) # [b, t', t], [b, t, d] -> [b, d, t']

    z_p = m_p + torch.randn_like(m_p) * torch.exp(logs_p) * noise_scale
    z = self.flow(z_p, y_mask, g=g, reverse=True)
    o = self.dec((z * y_mask)[:,:,:max_len], g=g)
    return o, attn, y_mask, (z, z_p, m_p, logs_p)

  def voice_conversion(self, y, y_lengths, sid_src, sid_tgt):
    assert self.n_speakers > 0, "n_speakers have to be larger than 0."
    g_src = self.emb_g(sid_src).unsqueeze(-1)
    g_tgt = self.emb_g(sid_tgt).unsqueeze(-1)
    z, m_q, logs_q, y_mask = self.enc_q(y, y_lengths, g=g_src)
    z_p = self.flow(z, y_mask, g=g_src)
    z_hat = self.flow(z_p, y_mask, g=g_tgt, reverse=True)
    o_hat = self.dec(z_hat * y_mask, g=g_tgt)
    return o_hat, y_mask, (z, z_p, z_hat)
"""

c:\Users\linka\anaconda3\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\linka\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\linka\anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


'\nclass SynthesizerTrn(nn.Module):\n\n  def __init__(self, \n    n_vocab,\n    spec_channels,\n    segment_size,\n    inter_channels,\n    hidden_channels,\n    filter_channels,\n    n_heads,\n    n_layers,\n    kernel_size,\n    p_dropout,\n    resblock, \n    resblock_kernel_sizes, \n    resblock_dilation_sizes, \n    upsample_rates, \n    upsample_initial_channel, \n    upsample_kernel_sizes,\n    n_speakers=0,\n    gin_channels=0,\n    use_sdp=True,\n    **kwargs):\n\n    super().__init__()\n    self.n_vocab = n_vocab\n    self.spec_channels = spec_channels\n    self.inter_channels = inter_channels\n    self.hidden_channels = hidden_channels\n    self.filter_channels = filter_channels\n    self.n_heads = n_heads\n    self.n_layers = n_layers\n    self.kernel_size = kernel_size\n    self.p_dropout = p_dropout\n    self.resblock = resblock\n    self.resblock_kernel_sizes = resblock_kernel_sizes\n    self.resblock_dilation_sizes = resblock_dilation_sizes\n    self.upsample_rates = up

In [2]:
# Define variables in the Class
n_vocab = 178 # len(symbols)
spec_channels = 1024 // 2 + 1,
segment_size = 8192 // 256
inter_channels = 192
hidden_channels = 192
filter_channels = 768
n_heads = 2
n_layers = 6
kernel_size = 3
p_dropout = 0.1
resblock = "1"
resblock_kernel_sizes = [3, 7, 11]
resblock_dilation_sizes = [[1, 3, 5], [1, 3, 5], [1, 3, 5]]
upsample_rates = [8, 8, 2, 2]
upsample_initial_channel = 512
upsample_kernel_sizes = [16, 16, 4, 4]
n_speakers = 0
gin_channels = 0
use_sdp = False

In [3]:
"""
self.enc_p = TextEncoder(n_vocab,
        inter_channels,
        hidden_channels,
        filter_channels,
        n_heads,
        n_layers,
        kernel_size,
        p_dropout)
self.dec = Generator(inter_channels, resblock, resblock_kernel_sizes, resblock_dilation_sizes, upsample_rates, upsample_initial_channel, upsample_kernel_sizes, gin_channels=gin_channels)
self.enc_q = PosteriorEncoder(spec_channels, inter_channels, hidden_channels, 5, 1, 16, gin_channels=gin_channels)
self.flow = ResidualCouplingBlock(inter_channels, hidden_channels, 5, 1, 4, gin_channels=gin_channels)

self.dp = DurationPredictor(hidden_channels, 256, 3, 0.5, gin_channels=gin_channels)

# we ignore emb_g = nn.Embedding(n_speakers) because it is used for multi-speaker TTS, which is not our case.
"""
# We need to define TextEncoder, Generator, PosteriorEncoder, ResidualCouplingBlock, DurationPredictor.


'\nself.enc_p = TextEncoder(n_vocab,\n        inter_channels,\n        hidden_channels,\n        filter_channels,\n        n_heads,\n        n_layers,\n        kernel_size,\n        p_dropout)\nself.dec = Generator(inter_channels, resblock, resblock_kernel_sizes, resblock_dilation_sizes, upsample_rates, upsample_initial_channel, upsample_kernel_sizes, gin_channels=gin_channels)\nself.enc_q = PosteriorEncoder(spec_channels, inter_channels, hidden_channels, 5, 1, 16, gin_channels=gin_channels)\nself.flow = ResidualCouplingBlock(inter_channels, hidden_channels, 5, 1, 4, gin_channels=gin_channels)\n\nself.dp = DurationPredictor(hidden_channels, 256, 3, 0.5, gin_channels=gin_channels)\n\n# we ignore emb_g = nn.Embedding(n_speakers) because it is used for multi-speaker TTS, which is not our case.\n'

![model](./static/fig_1a.png)

enc_p convert text into μ and σ (variational inference). Use mean and standard deviation to give variability to the model.

enc_q(posterior encoder)convert linear spectrogram into z

dec is decoder between z and waveform

flow convert z into f_{θ}(z), it is invertible, which means we can use a invert flow to convert f_{θ}(z') into z'

dp is DurationPredictor that predict the length of each phoneme

In [4]:
from einops import einsum

In [5]:
tensor1 = torch.randn(1, 192, 158)
tensor1 = rearrange(tensor1, 'b (h d) t -> b h t d', h=2)
tensor2 = torch.randn(1, 192, 158)
tensor2 = rearrange(tensor2, 'b (h d) t -> b h t d', h=2)
t1 = torch.matmul(tensor1, tensor2.transpose(-2, -1))
t2 = einsum(tensor1, tensor2, 'b h t1 d, b h t2 d -> b h t1 t2')
print(t1.shape)
print(t2.shape)
if(torch.all(t1.eq(t2))):
    print("True")

torch.Size([1, 2, 158, 158])
torch.Size([1, 2, 158, 158])
True


In [6]:


# Define some layers and submodels.
##################### This submodel is really hard #####################
class RelativeMultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.heads_share = True # Default is True, we only demonstrate True here.
        self.windows_size = 4 # Default is 4
        self.k_channels = hidden_channels // n_heads # multiheads attention channels is hidden_channels / n_heads

        # attention convolution layers
        self.conv_q = nn.Conv1d(hidden_channels, hidden_channels, 1)
        self.conv_k = nn.Conv1d(hidden_channels, hidden_channels, 1)
        self.conv_v = nn.Conv1d(hidden_channels, hidden_channels, 1)
        self.conv_o = nn.Conv1d(hidden_channels, hidden_channels, 1)

        self.drop = nn.Dropout(p_dropout)

        rel_stddev = self.k_channels ** -0.5
        self.emb_rel_k = nn.Parameter(torch.randn(1, self.windows_size * 2 + 1, self.k_channels) * rel_stddev) # First dimension is 1, because we want it share heads
        self.emb_rel_v = nn.Parameter(torch.randn(1, self.windows_size * 2 + 1, self.k_channels) * rel_stddev)

        # initialize conv weights
        nn.init.xavier_uniform_(self.conv_q.weight)
        nn.init.xavier_uniform_(self.conv_k.weight)
        nn.init.xavier_uniform_(self.conv_v.weight)
    
    def forward(self, x, c, attn_mask=None):
        q = self.conv_q(x)
        k = self.conv_k(c)
        v = self.conv_v(c)
        
        x, self.attn = self.attention(q, k, v, mask=attn_mask)

        x = self.conv_o(x)
        return x

    def attention(self, query, key, value, mask=None):
        # reshape [batch, hidden_channel, length] -> [batch, number of heads(2), length, hidden_channel / number of heads(d_k)]
        b, d, t_s, t_t = (*key.size(), query.size(2)) # t_s must be equal to t_t, t_s is the length of key, t_t is the length of query
        # Since it is self attention, so query = key = value, t_s = t_t
        query = rearrange(query, 'b (h d) t -> b h t d', h=2)
        key = rearrange(key, 'b (h d) t -> b h t d', h=2)
        value = rearrange(value, 'b (h d) t -> b h t d', h=2)

        # scores = torch.matmul(query / math.sqrt(self.k_channels), key.transpose(-2, -1)) # Query * Key
        scores = einsum(query, key, 'b h t1 d, b h t2 d -> b h t1 t2') # Query * Key^T
        key_relative_embeddings = self._get_relative_embeddings(self.emb_rel_k, t_s)
        rel_logits = self._matmul_with_relative_keys(query / math.sqrt(self.k_channels), key_relative_embeddings) #Query * Key embedding
        scores_local = self._relative_position_to_absolute_position(rel_logits)

        scores = scores + scores_local
        scores = scores.masked_fill(mask == 0, -1e4)

        p_attn = F.softmax(scores, dim=-1) # [b, n_h, t_t, t_s]
        p_attn = self.drop(p_attn)

        output = torch.matmul(p_attn, value) # attn * value
        relative_weights = self._absolute_position_to_relative_position(p_attn)
        value_relative_embeddings = self._get_relative_embeddings(self.emb_rel_v, t_s)
        output = output + self._matmul_with_relative_values(relative_weights, value_relative_embeddings) # attn * value embedding
        output = output.transpose(2, 3).contiguous().view(b, d, t_t) # [b, n_h, t_t, d_k] -> [b, d, t_t]
        return output, p_attn

    def _matmul_with_relative_values(self, x, y):
        """
        x: [b, h, l, m]
        y: [h or 1, m, d]
        ret: [b, h, l, d]
        b batch
        h heads
        l length
        m 2*length-1
        d hidden / heads

        """
        # ret = torch.matmul(x, y.unsqueeze(0))
        ret = einsum(x, y, "b h l m, s m d -> b h l d")
        return ret

    def _matmul_with_relative_keys(self, x, y):
        """
        x: [b, h, l, d]
        y: [h or 1, m, d]
        ret: [b, h, l, m]
        """
        # ret = torch.matmul(x, y.unsqueeze(0).transpose(-2, -1))
        ret = einsum(x, y, "b h l d, s m d -> b h l m")
        return ret

    def _get_relative_embeddings(self, relative_embeddings, length):
        max_relative_position = 2 * self.windows_size + 1
        # Pad first before slice to avoid using cond ops.
        pad_length = max(length - (self.windows_size + 1), 0)
        slice_start_position = max((self.windows_size + 1) - length, 0)
        slice_end_position = slice_start_position + 2 * length - 1
        if pad_length > 0:
            padded_relative_embeddings = F.pad(
                relative_embeddings,
                commons.convert_pad_shape([[0, 0], [pad_length, pad_length], [0, 0]])) # Just add zero to the second dimenision
                # For instance, if the original shape is [1, 9, 158], after padding, it will be [1, 9 + 2 * pad_length(add 0 to both pad_length before and after), 158]
        else:
            padded_relative_embeddings = relative_embeddings
        used_relative_embeddings = padded_relative_embeddings[:,slice_start_position:slice_end_position]
        return used_relative_embeddings
    

    def _relative_position_to_absolute_position(self, x): # Simply consider this method as a change of shape without losing any information
        """
        x: [b, h, l, 2*l-1]
        ret: [b, h, l, l]
        """
        batch, heads, length, _ = x.size()
        # Concat columns of pad to shift from relative to absolute indexing.
        x = F.pad(x, commons.convert_pad_shape([[0,0],[0,0],[0,0],[0,1]]))

        # Concat extra elements so to add up to shape (len+1, 2*len-1).
        x_flat = x.view([batch, heads, length * 2 * length])
        x_flat = F.pad(x_flat, commons.convert_pad_shape([[0,0],[0,0],[0,length-1]]))

        # Reshape and slice out the padded elements.
        x_final = x_flat.view([batch, heads, length+1, 2*length-1])[:, :, :length, length-1:]
        # print(x_final[0][0][0:3])
        return x_final

    def _absolute_position_to_relative_position(self, x): # Simply consider this method as a change of shape without losing any information
        """
        x: [b, h, l, l]
        ret: [b, h, l, 2*l-1]
        """
        batch, heads, length, _ = x.size()
        # padd along column
        x = F.pad(x, commons.convert_pad_shape([[0, 0], [0, 0], [0, 0], [0, length-1]]))
        x_flat = x.view([batch, heads, length**2 + length*(length -1)])
        # add 0's in the beginning that will skew the elements after reshape
        x_flat = F.pad(x_flat, commons.convert_pad_shape([[0, 0], [0, 0], [length, 0]]))
        x_final = x_flat.view([batch, heads, length, 2*length])[:,:,:,1:]
        return x_final

In [7]:
class LayerNorm(nn.Module):
  def __init__(self, channels, eps=1e-5):
    super().__init__()
    self.channels = channels
    self.eps = eps

    self.gamma = nn.Parameter(torch.ones(channels))
    self.beta = nn.Parameter(torch.zeros(channels))

  def forward(self, x):
    # x = x.transpose(1, -1)
    x = rearrange(x, 'b h l -> b l h')
    x = F.layer_norm(x, (self.channels,), self.gamma, self.beta, self.eps)
    return rearrange(x, 'b l h -> b h l')

In [17]:
class FFN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_1 = nn.Conv1d(hidden_channels, filter_channels, kernel_size)
    self.conv_2 = nn.Conv1d(filter_channels, hidden_channels, kernel_size)
    self.drop = nn.Dropout(p_dropout)

  def forward(self, x, x_mask):
    x = self.conv_1(self.padding(x * x_mask))
    x = torch.relu(x)
    x = self.drop(x)
    x = self.conv_2(self.padding(x * x_mask))
    return x * x_mask

  def padding(self, x):
    pad_l = (kernel_size - 1) // 2
    pad_r = kernel_size // 2
    padding = [[0, 0], [0, 0], [pad_l, pad_r]]
    x = F.pad(x, commons.convert_pad_shape(padding))
    return x

In [18]:
class Encoder(nn.Module):
  def __init__(self):
    super().__init__()
    
    self.drop = nn.Dropout(p_dropout)
    self.attn_layers = nn.ModuleList()
    self.norm_layers_1 = nn.ModuleList()
    self.ffn_layers = nn.ModuleList()
    self.norm_layers_2 = nn.ModuleList()
    for i in range(n_layers):
      self.attn_layers.append(RelativeMultiHeadAttention())
      self.norm_layers_1.append(LayerNorm(hidden_channels))
      self.ffn_layers.append(FFN())
      self.norm_layers_2.append(LayerNorm(hidden_channels))

  def forward(self, x, x_mask):
    attn_mask = x_mask.unsqueeze(2) * x_mask.unsqueeze(-1)
    x = x * x_mask
    for i in range(n_layers):
      y = self.attn_layers[i](x, x, attn_mask) # x(1, 192, 158), y(1, 192, 158)
      y = self.drop(y)
      x = self.norm_layers_1[i](x + y)

      y = self.ffn_layers[i](x, x_mask)
      y = self.drop(y)
      x = self.norm_layers_2[i](x + y)
    x = x * x_mask
    return x

In [19]:
class TextEncoder(nn.Module):
  def __init__(self,
      n_vocab,
      out_channels,
      hidden_channels,
      filter_channels,
      n_heads,
      n_layers,
      kernel_size,
      p_dropout):
    super().__init__()
    self.n_vocab = n_vocab
    self.out_channels = out_channels
    self.hidden_channels = hidden_channels
    self.filter_channels = filter_channels
    self.n_heads = n_heads
    self.n_layers = n_layers
    self.kernel_size = kernel_size
    self.p_dropout = p_dropout
    # The parameters in configs.

    self.emb = nn.Embedding(n_vocab, hidden_channels)
    nn.init.normal_(self.emb.weight, 0.0, hidden_channels**-0.5)

    self.encoder = Encoder()
    self.proj= nn.Conv1d(hidden_channels, out_channels * 2, 1)

  def forward(self, x, x_lengths):
    x = self.emb(x) * math.sqrt(self.hidden_channels) # [b, l, h]
    x = rearrange(x, 'b l h -> b h l') # [b, h, l]
    x_mask = torch.unsqueeze(commons.sequence_mask(x_lengths, x.size(2)), 1).to(x.dtype)

    x = self.encoder(x * x_mask, x_mask)

    stats = self.proj(x) * x_mask # Statistics, mean and log standard deviation.

    m, logs = torch.split(stats, self.out_channels, dim=1)
    return x, m, logs, x_mask

In [20]:
text_encoder = TextEncoder(n_vocab,
        inter_channels,
        hidden_channels,
        filter_channels,
        n_heads,
        n_layers,
        kernel_size,
        p_dropout)

In [21]:
# num parameters of textencoder
print(sum(p.numel() for p in text_encoder.parameters() if p.requires_grad)) 

6326784


In [22]:
input = torch.randint(0, 100, (1, 158))

In [23]:
print(input)

tensor([[83, 41, 37, 93, 68, 15, 66, 89, 65, 45, 24, 79, 19, 88, 86,  8, 89,  1,
          0, 17, 78, 93, 12, 47, 99, 32, 42, 71,  8, 23, 33, 16, 74, 31, 77, 84,
         73, 35, 24, 48, 65, 22, 20,  3, 48, 34, 93, 70, 75, 82, 29, 96, 20, 36,
         43, 97, 18, 54, 54, 95, 73, 35, 18, 52, 80, 53, 84, 77, 80, 39, 90, 95,
         55, 78, 17, 12,  4, 98, 86, 62, 75, 60,  9, 55, 64, 38, 62, 10, 92, 59,
         62, 16, 30, 44, 28,  8, 55, 52, 49, 27, 49, 13, 29, 90, 80, 66, 25, 46,
         88, 20, 72, 33, 17, 52, 51, 41, 96, 40, 21, 88, 85, 42, 92, 17, 96,  3,
         64, 53,  4,  3, 20,  9, 30, 35, 29, 97, 91, 36, 11, 26, 46, 80, 46, 70,
         17, 84, 41, 59, 84,  2, 79, 79,  3,  9, 70, 55, 26, 96]])


In [35]:
res = text_encoder(input, torch.tensor([158, 150, 140]))
print("text embedding:\n",res[0])
print("text embedding mean:\n",res[1])
print("text embedding stadard deviation:\n", res[2])
print("text embedding mask:\n", res[3])

text embedding:
 tensor([[[ 0.1528, -0.1475,  0.0824,  ...,  0.0665,  0.3523,  1.6038],
         [ 2.1663,  2.3274,  1.4338,  ..., -1.2666,  2.0447,  1.4563],
         [-0.3650, -0.0822,  0.0884,  ...,  1.1608,  0.0246,  0.7025],
         ...,
         [-0.4412,  0.9649,  2.2283,  ...,  1.3124,  0.3903, -0.3376],
         [ 1.0582,  0.7297,  2.0519,  ..., -0.2588, -1.3058, -1.3008],
         [-0.7863, -1.7969, -1.2143,  ...,  0.8865,  0.7399, -1.9964]],

        [[ 0.9681, -2.2346, -1.4794,  ..., -0.0000, -0.0000, -0.0000],
         [ 0.0702,  0.7630,  1.7258,  ...,  0.0000,  0.0000,  0.0000],
         [-0.1523,  0.3506,  0.6319,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-1.6298,  0.1538,  0.2883,  ...,  0.0000,  0.0000,  0.0000],
         [ 1.5832, -0.4347,  1.0357,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.2901, -1.4817, -0.1441,  ..., -0.0000, -0.0000, -0.0000]],

        [[-0.7879, -0.8448, -1.3634,  ..., -0.0000, -0.0000, -0.0000],
         [ 1.5008,  2.9067, 

In [36]:
####################### From the output above, you may know what does TextEncoding DO #######################